<a href="https://colab.research.google.com/github/vinodgaitonde/ProjIdeas/blob/main/Next_Word_Pred_Single_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pickle
import heapq

####################################
# Read the sample file
####################################
with open('./1661-half.txt', 'r', encoding='utf-8') as file:
    text = file.read()
#print('corpus length:', len(text))
#print(text[:100])

####################################
# Prepare the word index
####################################
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
# Get the word index
word_index = tokenizer.word_index

total_words = len(tokenizer.word_index) + 1

#print("tokenizer.word_index: ",len(tokenizer.word_index))
#print("total_words: ",total_words)
#print first 100 words
#print("word_index : ",tokenizer.word_index)

####################################
# Prepare input sequence & Find out the max Len
####################################
input_sequences = []
#j=0
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    #if(j<5):
    #    print(j, line)
    #    print(token_list)
    #    j+=1
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        #print ("\n", n_gram_sequence)
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(seq) for seq in input_sequences])

####################################
# Load model ./next_Word_19_Nov_v1_dropout.keras
####################################
model = load_model('./next_Word_19_Nov_v1_dropout.keras')

####################################
# Code to predict next N Words
####################################
def predict_next_n_words(seed_text, next_words):
    predicted_next_n_words = ""
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        preds = model.predict(token_list, verbose=0)
        #print(preds[0][303])
        #print(preds[0][304])
        #print("preds",preds)
        #print(preds.shape)
        predicted = np.argmax(preds, axis=-1)
        index=predicted[0]
        #print(predicted.shape)
        #print("predicted", predicted)

        output_word = ""

        # Get the word at the given index from the word_index
        output_word=list(word_index.keys())[list(word_index.values()).index(index)]

        # print("predicted", output_word)
        """
        for word, index in tokenizer.word_index.items():
            print("word",word)
            print("index",index)
            if index == predicted:
                output_word = word
                break
        """
        seed_text += " " + output_word
        predicted_next_n_words += " " + output_word

       #print(save_seed_text)
        #print(output_word)
    return predicted_next_n_words

####################################
# Code to predict Best N options for next Word
####################################
def predict_next_word_top_options(seed_text, num_words):
    predicted_words = []
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    preds = model.predict(token_list,verbose=0)[0]
    # Get the indices of the  n largest elements from an iterable.
    # find out the top predictions from preds
    top_indices = heapq.nlargest(num_words, range(len(preds)), preds.take)
    #print(top_indices)
    # Get the corresponding words from the unique word index
    predicted_words = [list(word_index.keys())[list(word_index.values()).index(i)] for i in top_indices]
    return predicted_words

####################################
# Loop for uer input and response
####################################
while(True):
  text = input("Enter your line: ")

  if text == "0":
      print("Execution completed.....")
      break

  else:
      try:
          text = text.split(" ")
          #text = text[-3:]
          #print(text)

          predicted_words = predict_next_word_top_options(text, 5)
          #print(f"Input Text: {text}")
          print(f"Predicted Next Words: {predicted_words}\n")

      except Exception as e:
        print("Error occurred: ",e)
        continue

SyntaxError: invalid syntax (<ipython-input-9-a7bbd5626720>, line 138)